In [3]:
import pandas as pd
import xlsxwriter


def create_real_estate_excel_v14(filename='房地拆價試算表_右側說明版.xlsx'):
    """
    V14.0 更新重點：
    1. 【版面重構】使用指南移至「右側」，輸入與查看說明可同時進行。
    2. 【數據格式】房地比改為小數點格式 (0.00)，例如 0.23。
    3. 【內容完善】詳細收錄「未稅/含稅」拆分邏輯與驗算說明。
    """

    writer = pd.ExcelWriter(filename, engine='xlsxwriter')
    workbook = writer.book

    FONT_NAME = 'Microsoft JhengHei'

    # 建立兩個 Sheet
    sheet_calc = workbook.add_worksheet('拆價與財交稅試算')
    sheet_ref = workbook.add_worksheet('113年度稅率參照表')

    # ==========================================
    # PART 1: 主計算頁面 (Main Calculation Sheet)
    # ==========================================
    setup_calculation_sheet_v14(workbook, sheet_calc, FONT_NAME)

    # ==========================================
    # PART 2: 稅率參照頁面 (Reference Sheet)
    # ==========================================
    setup_reference_sheet(workbook, sheet_ref, FONT_NAME)

    writer.close()
    print(f"檔案已生成：{filename}")


def setup_calculation_sheet_v14(workbook, worksheet, font_name):
    """設定主計算頁面 - V14 右側說明版"""

    worksheet.hide_gridlines(2)

    # --- 樣式定義 ---
    # 標題
    fmt_title = workbook.add_format({
        'font_name': font_name, 'bold': True, 'font_size': 16,
        'align': 'left', 'valign': 'vcenter', 'font_color': '#333333'
    })

    # 輸入區 (黃底)
    fmt_input_label = workbook.add_format({'font_name': font_name, 'font_size': 11, 'bold': True, 'align': 'left',
                                          'valign': 'vcenter', 'fg_color': '#FFF2CC', 'font_color': '#7F6000', 'border': 1, 'border_color': '#BF9000'})
    fmt_input_val = workbook.add_format({'font_name': font_name, 'font_size': 14, 'bold': True, 'align': 'right', 'valign': 'vcenter',
                                        'num_format': '#,##0', 'fg_color': '#FFFFE0', 'font_color': '#000000', 'border': 1, 'border_color': '#BF9000'})
    fmt_input_rate = workbook.add_format({'font_name': font_name, 'font_size': 14, 'bold': True, 'align': 'right',
                                         'valign': 'vcenter', 'fg_color': '#FFFFE0', 'font_color': '#000000', 'border': 1, 'border_color': '#BF9000'})

    # 結果表格
    fmt_row_label = workbook.add_format({'font_name': font_name, 'bold': True, 'align': 'center',
                                        'valign': 'vcenter', 'bg_color': '#F2F2F2', 'font_color': '#333333', 'border': 1})
    fmt_col_green = workbook.add_format({'font_name': font_name, 'bold': True, 'align': 'center',
                                        'valign': 'vcenter', 'fg_color': '#E2EFDA', 'font_color': '#375623', 'border': 1})
    fmt_col_blue = workbook.add_format({'font_name': font_name, 'bold': True, 'align': 'center',
                                       'valign': 'vcenter', 'fg_color': '#DDEBF7', 'font_color': '#1F4E78', 'border': 1})

    fmt_res_green = workbook.add_format({'font_name': font_name, 'font_size': 13, 'align': 'right', 'valign': 'vcenter',
                                        'num_format': '#,##0', 'bg_color': '#FAFAFA', 'font_color': '#375623', 'border': 1, 'bold': True})
    fmt_res_blue = workbook.add_format({'font_name': font_name, 'font_size': 13, 'align': 'right', 'valign': 'vcenter',
                                       'num_format': '#,##0', 'bg_color': '#FAFAFA', 'font_color': '#1F4E78', 'border': 1, 'bold': True})

    # 財交稅與房地比
    fmt_tax_header = workbook.add_format({'font_name': font_name, 'bold': True, 'align': 'center',
                                         'valign': 'vcenter', 'fg_color': '#FCE4D6', 'font_color': '#C65911', 'border': 1})
    fmt_tax_res = workbook.add_format({'font_name': font_name, 'font_size': 13, 'align': 'right', 'valign': 'vcenter',
                                      'num_format': '#,##0', 'bg_color': '#FAFAFA', 'font_color': '#C65911', 'border': 1, 'bold': True})

    # ★ 重點修改：房地比顯示格式 (0.00)
    fmt_ratio_res = workbook.add_format({
        'font_name': font_name, 'font_size': 13, 'align': 'right', 'valign': 'vcenter',
        'num_format': '0.00',  # 顯示為 0.xx
        'bg_color': '#FAFAFA', 'font_color': '#333333', 'border': 1, 'bold': True
    })

    # 驗算與提示
    fmt_check_label = workbook.add_format({'font_name': font_name, 'font_size': 10, 'align': 'right',
                                          'valign': 'vcenter', 'font_color': '#595959', 'bg_color': '#F9F9F9', 'border': 1})
    fmt_check_val = workbook.add_format({'font_name': font_name, 'font_size': 10, 'align': 'right',
                                        'valign': 'vcenter', 'num_format': '#,##0', 'font_color': '#595959', 'bg_color': '#F9F9F9', 'border': 1})
    fmt_calc_basis = workbook.add_format({'font_name': font_name, 'font_size': 11, 'align': 'center',
                                         'valign': 'vcenter', 'font_color': '#FFFFFF', 'bg_color': '#595959', 'bold': True, 'border': 1})
    fmt_link = workbook.add_format(
        {'font_name': font_name, 'font_size': 10, 'valign': 'vcenter', 'color': 'blue', 'underline': True})
    fmt_hint = workbook.add_format(
        {'font_name': font_name, 'font_size': 10, 'valign': 'vcenter', 'color': 'gray'})

    # ★ 右側說明專用樣式
    fmt_right_guide = workbook.add_format({
        'font_name': font_name, 'font_size': 10, 'color': '#404040',
        'text_wrap': True, 'valign': 'top', 'align': 'left',
        'bg_color': '#FDFDFD', 'border': 1, 'border_color': '#D9D9D9'
    })
    fmt_guide_title = workbook.add_format({
        'font_name': font_name, 'font_size': 11, 'bold': True, 'color': 'white',
        'bg_color': '#70AD47', 'align': 'center', 'valign': 'vcenter', 'border': 1
    })

    # --- 佈局 (Columns) ---
    worksheet.set_column('A:A', 2)   # 左邊距
    worksheet.set_column('B:B', 28)  # 標籤
    worksheet.set_column('C:C', 22)  # 數值
    worksheet.set_column('D:D', 20)  # 連結與單位
    worksheet.set_column('E:E', 2)   # 間隔
    worksheet.set_column('F:H', 12)  # 右側說明區寬度 (F, G, H)

    # Rows height
    worksheet.set_row(0, 35)
    for r in range(3, 8):
        worksheet.set_row(r, 30)  # 輸入區高度

    # 標題
    worksheet.merge_range('B1:D1', '房地拆價與財交稅試算表', fmt_title)

    # ==========================================
    # 1. 左側：輸入與計算區
    # ==========================================

    # --- 輸入區 ---
    start_row = 3
    items = [
        ("1. 土地公告現值 (L)", 0, "元"),
        ("2. 建物評定/公告現值 (B)", 0, "元"),
        ("3. 成交總價 (T)", 0, "元"),
        ("4. 車位價格 (若無則免填)", 0, "元"),
    ]

    for i, (label, default, hint) in enumerate(items):
        r = start_row + i
        worksheet.write(r, 1, label, fmt_input_label)
        worksheet.write(r, 2, default, fmt_input_val)
        worksheet.write(r, 3, hint, fmt_hint)

    # 稅率輸入
    r_rate = start_row + 4
    worksheet.write(r_rate, 1, "5. 所得標準率 (輸入10或0.1)", fmt_input_label)
    worksheet.write(r_rate, 2, 0, fmt_input_rate)
    worksheet.write_url(r_rate, 3, "internal:'113年度稅率參照表'!A1",
                        string="查詢113年稅率表 >>", cell_format=fmt_link)

    # 定義參照
    L, B, T, C, Rate = "C4", "C5", "C6", "C7", "C8"

    # --- 拆價結果 ---
    r = 9
    worksheet.merge_range(
        r, 1, r, 3, f'="計算基準： "&TEXT({T},"#,##0")&" - "&TEXT({C},"#,##0")&" = "&TEXT({T}-{C},"#,##0")', fmt_calc_basis)

    r += 1
    worksheet.write(r, 1, "拆分項目", fmt_row_label)
    worksheet.write(r, 2, "一般個人 (未稅)", fmt_col_green)
    worksheet.write(r, 3, "法人/公司 (含稅)", fmt_col_blue)

    r += 1
    worksheet.write(r, 1, "土地拆價", fmt_row_label)
    worksheet.write_formula(
        r, 2, f"=IF({T}=0,0, ({T}-{C}) * ({L}/({L}+{B})))", fmt_res_green)
    worksheet.write_formula(
        r, 3, f"=IF({T}=0,0, ({T}-{C}) * ({L}/({L}+{B}*1.05)))", fmt_res_blue)

    r += 1
    worksheet.write(r, 1, "建物拆價", fmt_row_label)
    worksheet.write_formula(
        r, 2, f"=IF({T}=0,0, ({T}-{C}) * ({B}/({L}+{B})))", fmt_res_green)
    worksheet.write_formula(
        r, 3, f"=IF({T}=0,0, ({T}-{C}) * (({B}*1.05)/({L}+{B}*1.05)))", fmt_res_blue)

    # 驗算
    row_land, row_bldg = r-1, r
    r += 1
    worksheet.write(r, 1, "驗算總價 (含車位)", fmt_check_label)
    worksheet.write_formula(
        r, 2, f"=C{row_land+1}+C{row_bldg+1}+{C}", fmt_check_val)
    worksheet.write_formula(
        r, 3, f"=D{row_land+1}+D{row_bldg+1}+{C}", fmt_check_val)

    # --- 財交稅結果 ---
    r += 2
    worksheet.merge_range(r, 1, r, 3, "【舊制財交稅試算結果】", fmt_tax_header)

    r += 1
    worksheet.write(r, 1, "房地比 (建物/總現值)", fmt_row_label)
    # 格式指定為 '0.00'
    worksheet.merge_range(
        r, 2, r, 3, f"=IF(({L}+{B})=0, 0, {B}/({L}+{B}))", fmt_ratio_res)

    r += 1
    worksheet.write(r, 1, "出售房屋財交所得額", fmt_row_label)
    worksheet.merge_range(
        r, 2, r, 3, f"=ROUND({B} * IF({Rate}>=1, {Rate}/100, {Rate}), 0)", fmt_tax_res)

    # ==========================================
    # 2. 右側：詳細使用指南 (F欄開始)
    # ==========================================
    # 合併 F3 到 H18 顯示說明文字

    guide_header = "【使用提示與計算邏輯說明】"

    guide_text = (
        "1. 拆算核心邏輯：\n"
        "   先將 [成交總價] 扣除 [車位價格] 得到房地淨額，再依下列比例拆分。\n\n"
        "2. 兩種情境的比例計算方式：\n"
        "   ● 一般個人 (未稅)：\n"
        "     不考慮營業稅，直接依公告現值比例拆分。\n"
        "     → 土地比例公式：土地現值 ÷ (土地現值 + 建物現值)\n"
        "     → 建物比例公式：建物現值 ÷ (土地現值 + 建物現值)\n\n"
        "   ● 法人/公司 (含稅)：\n"
        "     因法人出售建物需開立含 5% 營業稅發票，故建物價值需加權。\n"
        "     → 土地比例公式：土地現值 ÷ (土地現值 + 建物現值 × 1.05)\n"
        "     → 建物比例公式：(建物現值 × 1.05) ÷ (土地現值 + 建物現值 × 1.05)\n"
        "     (註：此法會使建物的拆價金額略高，以反映含稅價值)\n\n"
        "3. 驗算確認：\n"
        "   本表下方已將拆分後的房地與車位加回，總額應等於成交總價。\n\n"
        "4. 房地比與財交稅：\n"
        "   房地比格式顯示為小數點 (如 0.32)，方便直接填寫申報書。"
    )

    worksheet.merge_range('F3:H3', guide_header, fmt_guide_title)
    worksheet.merge_range('F4:H18', guide_text, fmt_right_guide)


def setup_reference_sheet(workbook, worksheet, font_name):
    """建立詳細的稅率參照表 (維持原邏輯)"""
    worksheet.set_column('A:A', 15)
    worksheet.set_column('B:B', 40)
    worksheet.set_column('C:C', 15)
    worksheet.set_column('D:D', 30)

    fmt_header = workbook.add_format({'font_name': font_name, 'bold': True,
                                     'font_size': 12, 'bg_color': '#4472C4', 'font_color': 'white', 'border': 1})
    fmt_sub_header = workbook.add_format(
        {'font_name': font_name, 'bold': True, 'font_size': 11, 'bg_color': '#D9E1F2', 'font_color': 'black', 'border': 1})
    fmt_text = workbook.add_format(
        {'font_name': font_name, 'font_size': 10, 'text_wrap': True, 'valign': 'top', 'border': 1})
    fmt_rate = workbook.add_format({'font_name': font_name, 'font_size': 10, 'align': 'center',
                                   'valign': 'top', 'border': 1, 'bold': True, 'color': '#C00000'})

    headers = ["縣市/類別", "行政區或條件", "標準/稅率", "備註"]
    for col, h in enumerate(headers):
        worksheet.write(0, col, h, fmt_header)

    row = 1

    # 豪宅
    high_value_data = [
        ("臺北市", "房地總成交金額 ≧ 6,000萬元 或 單價 ≧ 120萬/坪",
         "按實際所得核實", "無法舉證成本者，採收入 × 房地比 × 20%"),
        ("新北市", "房地總成交金額 ≧ 4,000萬元 或 單價 ≧ 75萬/坪", "按實際所得核實", "同上"),
        ("桃園/竹縣市/中/南/高", "房地總成交金額 ≧ 3,000萬元 或 單價 ≧ 50萬/坪", "按實際所得核實", "同上"),
        ("其他地區", "房地總成交金額 ≧ 2,200萬元 或 單價 ≧ 35萬/坪", "按實際所得核實", "同上"),
    ]

    worksheet.merge_range(
        row, 0, row, 3, "【第一類：高總價/豪宅 (採實際成交金額計算)】", fmt_sub_header)
    row += 1
    for item in high_value_data:
        worksheet.write(row, 0, item[0], fmt_text)
        worksheet.write(row, 1, item[1], fmt_text)
        worksheet.write(row, 2, item[2], fmt_rate)
        worksheet.write(row, 3, item[3], fmt_text)
        row += 1

    # 一般案件
    row += 1
    worksheet.merge_range(
        row, 0, row, 3, "【第二類：一般案件 (房屋評定現值 × 稅率)】 - 財政部 114.2.19 公告", fmt_sub_header)
    row += 1

    standard_data = [
        ("臺北市", "中山區、松山區、中正區、大安區、信義區", "48%"),
        ("臺北市", "士林區、內湖區、大同區、南港區", "46%"),
        ("臺北市", "北投區、萬華區、文山區", "45%"),
        ("新北市", "板橋、永和、中和、三重、新店、蘆洲、新莊、土城", "44%"),
        ("新北市", "林口區", "42%"),
        ("新北市", "汐止區、樹林區", "41%"),
        ("新北市", "泰山區", "36%"),
        ("新北市", "五股區、三峽區", "34%"),
        ("新北市", "淡水區", "29%"),
        ("新北市", "八里區", "27%"),
        ("新北市", "鶯歌區", "24%"),
        ("新北市", "深坑區", "23%"),
        ("新北市", "金山區、三芝區、萬里區", "19%"),
        ("新北市", "瑞芳區", "18%"),
        ("新北市", "石碇、烏來、平溪、坪林、石門、雙溪、貢寮", "14%"),
        ("桃園市", "桃園區、中壢區", "33%"),
        ("桃園市", "龜山區、蘆竹區、八德區", "32%"),
        ("臺中市", "西屯區、南屯區", "35%"),
        ("臺中市", "北屯區、西區", "32%"),
        ("臺南市", "東區", "30%"),
        ("高雄市", "左營、鼓山、三民", "37%"),
        ("高雄市", "苓雅、前鎮", "35%"),
        ("新竹市", "全區", "32%"),
        ("新竹縣", "竹北市", "37%"),
    ]

    for item in standard_data:
        worksheet.write(row, 0, item[0], fmt_text)
        worksheet.write(row, 1, item[1], fmt_text)
        worksheet.write(row, 2, item[2], fmt_rate)
        worksheet.write(row, 3, "", fmt_text)
        row += 1


if __name__ == "__main__":
    create_real_estate_excel_v14()

檔案已生成：房地拆價試算表_右側說明版.xlsx
